# 소방청 - 광주지역 검색

In [100]:
!apt-get update > /dev/null 2>&1
!pip install selenium > /dev/null 2>&1
!apt install chromium-chromedriver > /dev/null 2>&1

In [101]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
from bs4 import BeautifulSoup

In [102]:
options = webdriver.ChromeOptions()
options.add_argument('--headless') # 화면없이 실행
options.add_argument('--no-sandbox')
options.add_argument('--single-process')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)

In [103]:
import warnings
warnings.filterwarnings('ignore')

In [104]:
url = 'https://www.nfa.go.kr/nfa/introduce/status/firestationidfo/?searchDistance=10&searchMode=distance&myX=35.1493479&myY=126.9218479&searchKeyword='
driver.get(url)

In [105]:
# 입력창에 광주광역시를 입력하고 검색버튼을 누르는 동작
driver.find_element(By.ID, 'searchKeyword').send_keys('광주광역시')
driver.find_element(By.ID, 'fsSearchBtn').click()

### 이동 후에는 BeautifulSoup 이용하여 정보를 획득

In [106]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
lis = soup.select('.stations-list > li') #클래스니까 꼭 . 붙이기
len(lis)

10

In [107]:
li = lis[0]
name = li.select_one('.title').get_text().strip()
name

'광산소방서_광주-평동-119 안전센터'

In [108]:
addr = li.find('address').string.strip()
addr

'광주광역시 광산구 월전동 1251'

In [109]:
tel = li.select_one('.tel').get_text().strip()
tel

'전화공사중'

In [110]:
lines = []
for li in lis:
  name = li.select_one('.title').get_text().strip()
  addr = li.find('address').string.strip()
  tel = li.select_one('.tel').get_text().strip()
  lines.append([name, addr, tel])

### 2페이지로 이동

In [111]:
# 2쪽 부분에 copy Xpath
# 화면이 바뀜 
driver.find_element(By.XPATH, '//*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[4]/a').click()

In [112]:
# 페이지 이동 후 정보 가져오기 
soup = BeautifulSoup(driver.page_source, 'html.parser')
lis = soup.select('.stations-list > li') #클래스니까 꼭 . 붙이기
len(lis)

10

### 다음페이지로 이동

In [113]:
driver.find_element(By.CSS_SELECTOR, '.next_page').click()

### 전체

In [114]:
# 검색결과 건수 획득
strongs = soup.select('.stations-local-after > p > strong')
strongs[1].get_text()

'29건'

In [115]:
count = int(strongs[1].get_text().replace('건','').strip())
count


29

In [116]:
#루프를 3번 돈다 (10개씩 돌아서?)
import math
count = math.ceil(count / 10)
count

3

### 최종 코드

In [117]:
# 원래의 위치로 만들기
driver.get(url)
time.sleep
driver.find_element(By.ID, 'searchKeyword').send_keys('광주광역시')
driver.find_element(By.ID, 'fsSearchBtn').click()

In [118]:
strongs = soup.select('.stations-local-after > p > strong')
count = int(strongs[1].get_text().replace('건','').strip())
count = math.ceil(count / 10)

In [119]:
lines = []
for i in range(count):
    if i >=1 and i%2 ==0:
        driver.find_element(By.CSS_SELECTOR, '.next_page').click()
        time.sleep(2) #이동을 하고 나서 약간의 시간을 줘야함
    if i >=1 and i%1 ==1:
        driver.find_element(By.XPATH, '//*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[4]/a').click()
        time.sleep(2)    

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    lis = soup.select('.stations-list > li') #클래스니까 꼭 . 붙이기
    for li in lis:
        name = li.select_one('.title').get_text().strip()
        addr = li.find('address').string.strip()
        tel = li.select_one('.tel').get_text().strip()
        lines.append([name, addr, tel])

In [120]:
len(lines)

29

In [121]:
driver.close()

In [123]:
df = pd.DataFrame(lines, columns=['관서명','주소','전화번호'])
df.tail()

,관서명,주소,전화번호
24,광산소방서,광주광역시 광산구 하남산단1번로 13(하남동),062-613-8813
25,남부소방서,광주광역시 남구 송암로58번길 13(송하동),062-613-8613
26,동부소방서,광주광역시 동구 제봉로 210(대인동),062-613-8413
27,북부소방서,광주광역시 북구 서하로 290(오치동),062-613-8713
28,서부소방서,광주광역시 서구 화운로 197(화정동),062-613-8513
